In [1]:
import os 
os.chdir("C:\\Users\\86182\\Desktop")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.callbacks import History

from wordcloud import WordCloud, STOPWORDS

from bs4 import BeautifulSoup
import re,string,unicodedata
from string import punctuation

In [3]:
true = pd.read_csv("True.csv")
false = pd.read_csv("Fake.csv")

In [4]:
true['category'] = 1
false['category'] = 0
df = pd.concat([true,false])
df['text'] = df['text'] + " " + df['title']
del df['date']
del df['title']

In [5]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [6]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['text']=df['text'].apply(denoise_text)

D:\anaconda\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [7]:
del df['subject']

训练模型

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['category'], test_size = 0.3, random_state= 0)

In [9]:
from sklearn.svm import SVC
import lightgbm as lgb

In [10]:
def get_prediction(vectorizer, classifier, X_train, X_test, y_train, y_test):
    pipe = Pipeline([('vector', vectorizer),
                    ('model', classifier)])
    model = pipe.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Accuarcy: {}".format(round(accuracy_score(y_test, y_pred)*100,2)))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix: \n", cm)
    print("Classification Report: \n", classification_report(y_test, y_pred))

In [11]:
classifiers = [lgb.LGBMClassifier()]
for classifier in classifiers:
    print("\n\n", classifier)
    print("***********Usng TFIDF Vectorizer****************")
    get_prediction(TfidfVectorizer(max_features=5000,stop_words='english'), classifier, X_train, X_test, y_train, y_test)



 LGBMClassifier()
***********Usng TFIDF Vectorizer****************
[LightGBM] [Info] Number of positive: 15016, number of negative: 16412
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.944234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 754555
[LightGBM] [Info] Number of data points in the train set: 31428, number of used features: 5000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.477791 -> initscore=-0.088896
[LightGBM] [Info] Start training from score -0.088896
Accuarcy: 99.74
Confusion Matrix: 
 [[7049   20]
 [  15 6386]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7069
           1       1.00      1.00      1.00      6401

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [12]:
classifiers = [SVC()]
for classifier in classifiers:
    print("\n\n", classifier)
    print("***********Usng TFIDF Vectorizer****************")
    get_prediction(TfidfVectorizer(max_features=5000,stop_words='english'), classifier, X_train, X_test, y_train, y_test)



 SVC()
***********Usng TFIDF Vectorizer****************
Accuarcy: 99.28
Confusion Matrix: 
 [[7012   57]
 [  40 6361]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7069
           1       0.99      0.99      0.99      6401

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [13]:
from sklearn.svm import LinearSVC

In [15]:
classifiers = [LinearSVC()]
for classifier in classifiers:
    print("\n\n", classifier)
    print("***********Usng TFIDF Vectorizer****************")
    get_prediction(TfidfVectorizer(max_features=5000,stop_words='english'), classifier, X_train, X_test, y_train, y_test)



 LinearSVC()
***********Usng TFIDF Vectorizer****************
Accuarcy: 99.53
Confusion Matrix: 
 [[7031   38]
 [  25 6376]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.99      1.00      7069
           1       0.99      1.00      1.00      6401

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470

